# S&P 100 Data Estimates - Single Stock

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import datetime as dt
import quandl

## Data 

In [2]:
datadir='../../data/'
assets=pd.read_csv(datadir + 'SP100.csv', comment='#').set_index('Symbol')
QUANDL={
    'authtoken':"6XyApK2BBj_MraQg2TMD",
    'start_date':dt.date(2016, 1, 1),
    'end_date':dt.date(2018, 12, 31)
}
RISK_FREE_SYMBOL = "USDOLLAR"
data={}

#### Download loop

If it stops because of Quandl error codes 503 or 504, try re-running it (it won't download data already downloaded). If Quandl complains about the speed of requests, try adding sleep time.

In [3]:
# Download assets' data
from time import sleep

def to_quandl_eod_ticker(ticker):
    '''
    Converts ticker to format in Quandl EOD dataset
    '''
    if 'USDOLLAR' not in ticker:
        return 'EOD/' + ticker.replace('.', '_')
    else:
        return 'FRED/DTB3'

# Construct a data dictionary: {ticker: pd.DataFrame(price/volume)}
for ticker in assets.index:
    if ticker in data:
        continue
    print('downloading %s from %s to %s' %(ticker, QUANDL['start_date'], QUANDL['end_date']))
    try:
        data[ticker] = quandl.get(to_quandl_eod_ticker(ticker), **QUANDL)
    except quandl.NotFoundError:
        print('\tInvalid asset code')

downloading AAPL from 2016-01-01 to 2018-12-31
downloading ABBV from 2016-01-01 to 2018-12-31
downloading ABT from 2016-01-01 to 2018-12-31
downloading ACN from 2016-01-01 to 2018-12-31
downloading ADBE from 2016-01-01 to 2018-12-31
downloading AGN from 2016-01-01 to 2018-12-31
downloading AIG from 2016-01-01 to 2018-12-31
downloading ALL from 2016-01-01 to 2018-12-31
downloading AMGN from 2016-01-01 to 2018-12-31
downloading AMZN from 2016-01-01 to 2018-12-31
downloading AXP from 2016-01-01 to 2018-12-31
downloading BA from 2016-01-01 to 2018-12-31
downloading BAC from 2016-01-01 to 2018-12-31
downloading BIIB from 2016-01-01 to 2018-12-31
downloading BK from 2016-01-01 to 2018-12-31
downloading BKNG from 2016-01-01 to 2018-12-31
downloading BLK from 2016-01-01 to 2018-12-31
downloading BMY from 2016-01-01 to 2018-12-31
downloading BRK.B from 2016-01-01 to 2018-12-31
downloading C from 2016-01-01 to 2018-12-31
downloading CAT from 2016-01-01 to 2018-12-31
downloading CELG from 2016-01

#### Computation 

In [4]:
keys=[el for el in assets.index if not el in (set(assets.index)-set(data.keys()))]

def select_first_valid_column(df, columns):
    for column in columns:
        if column in df.columns:
            return df[column]

# extract prices
prices=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Adj. Close", "Close", "Value"])
                                              for k in keys])))

#compute sigmas
open_price=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Open"]) for k in keys])))
close_price=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Close"]) for k in keys])))
sigmas = np.abs(np.log(open_price.astype(float))-np.log(close_price.astype(float)))

# extract volumes
volumes=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Adj. Volume", "Volume"])
                                               for k in keys])))

# fix risk free
prices[RISK_FREE_SYMBOL]=10000*(1 + prices[RISK_FREE_SYMBOL]/(100*250)).cumprod()

#### Filtering 

In [5]:
# filter NaNs - threshold at 2% missing values
bad_assets = prices.columns[prices.isnull().sum()>len(prices)*0.02]
if len(bad_assets):
    print('Assets %s have too many NaNs, removing them' % bad_assets)

prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]

nassets=prices.shape[1]

# days on which many assets have missing values
bad_days1=sigmas.index[sigmas.isnull().sum(1) > nassets*.9]
bad_days2=prices.index[prices.isnull().sum(1) > nassets*.9]
bad_days3=volumes.index[volumes.isnull().sum(1) > nassets*.9]
bad_days=pd.Index(set(bad_days1).union(set(bad_days2)).union(set(bad_days3))).sort_values()
print ("Removing these days from dataset:")
print(pd.DataFrame({'nan price':prices.isnull().sum(1)[bad_days],
                    'nan volumes':volumes.isnull().sum(1)[bad_days],
                    'nan sigmas':sigmas.isnull().sum(1)[bad_days]}))

prices=prices.loc[~prices.index.isin(bad_days)]
sigmas=sigmas.loc[~sigmas.index.isin(bad_days)]
volumes=volumes.loc[~volumes.index.isin(bad_days)]

# extra filtering
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))
prices=prices.fillna(method='ffill')
sigmas=sigmas.fillna(method='ffill')
volumes=volumes.fillna(method='ffill')
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))

Assets Index(['DD', 'DOW'], dtype='object') have too many NaNs, removing them
Removing these days from dataset:
Empty DataFrame
Columns: [nan price, nan volumes, nan sigmas]
Index: []
          remaining nan price  remaining nan volumes  remaining nan sigmas
AAPL                        0                      0                     0
ABBV                        0                      0                     0
ABT                         0                      0                     0
ACN                         0                      0                     0
ADBE                        0                      0                     0
...                       ...                    ...                   ...
WBA                         0                      0                     0
WFC                         0                      0                     0
WMT                         0                      0                     0
XOM                         0                      0              

#### Save 

In [6]:
# make volumes in dollars
volumes = volumes*prices

# compute returns
returns = (prices.diff()/prices.shift(1)).fillna(method='ffill').iloc[1:]

bad_assets = returns.columns[((-.5>returns).sum()>0)|((returns > 2.).sum()>0)]
if len(bad_assets):
    print('Assets %s have dubious returns, removed' % bad_assets)
    
prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]
returns = returns.loc[:,~returns.columns.isin(bad_assets)]

# remove USDOLLAR except from returns
prices = prices.iloc[:,:-1]
sigmas = sigmas.iloc[:,:-1]
volumes = volumes.iloc[:,:-1]


# save data
prices.to_csv(datadir+'ss_prices.csv.gz', compression='gzip', float_format='%.3f')
volumes.to_csv(datadir+'ss_volumes.csv.gz', compression='gzip', float_format='%d')
returns.to_csv(datadir+'ss_returns.csv.gz', compression='gzip', float_format='%.3e')
sigmas.to_csv(datadir+'ss_sigmas.csv.gz', compression='gzip', float_format='%.3e')

## Estimates 

In [7]:
print("Typical variance of returns: %g"%returns.var().mean())

Typical variance of returns: 0.000212859


In [8]:
return_estimate = returns.ewm(alpha=0.1, min_periods=60).mean().shift(1).dropna()
return_estimate

,AAPL,ABBV,ABT,ACN,ADBE,AGN,AIG,ALL,AMGN,AMZN,...,UPS,USB,UTX,V,VZ,WBA,WFC,WMT,XOM,USDOLLAR
Date,,,,,,,,,,,,,,,,,,,,,
2016-04-01,0.004391,0.001582,0.003592,0.006434,0.004440,-0.004225,0.001798,0.001986,0.001931,0.004005,...,0.002473,0.000073,0.002533,0.004065,0.002306,0.002850,-0.001636,0.001376,0.000375,0.000011
2016-04-04,0.004870,0.001949,0.004094,0.006614,0.005448,-0.003779,0.003174,0.003066,0.004564,0.004424,...,0.001732,0.000337,0.002149,0.005112,0.001945,0.005490,-0.001286,0.002072,-0.000418,0.000011
2016-04-05,0.005412,0.004876,0.003993,0.005220,0.004208,0.000133,0.001962,0.001893,0.004432,0.003093,...,0.001491,0.000254,0.002234,0.004910,0.002511,0.004490,-0.001054,0.001922,-0.000134,0.000011
2016-04-06,0.003689,0.003255,0.002930,0.003510,0.003437,-0.014672,0.000441,0.000312,0.002881,0.001593,...,0.000425,-0.001322,0.001601,0.003312,0.001652,0.000618,-0.002992,0.001063,-0.001265,0.000011
2016-04-07,0.004368,0.005238,0.005042,0.003632,0.004400,-0.009736,0.001198,0.000776,0.006985,0.004156,...,0.001105,-0.001090,0.001952,0.004593,0.000432,0.000952,-0.001492,0.001540,0.000201,0.000010
2016-04-08,0.001748,0.003727,0.003514,0.001025,0.002260,-0.010292,-0.000772,-0.000422,0.004856,0.001969,...,0.000344,-0.003453,0.002275,0.003327,-0.002455,-0.002002,-0.003737,0.000197,-0.000948,0.000010
2016-04-11,0.001683,0.001953,0.002599,0.001574,0.002343,-0.011338,-0.000600,0.000466,0.003785,0.002309,...,0.000618,-0.002390,0.002474,0.003574,-0.001862,-0.001715,-0.003064,-0.000057,0.000167,0.000010
2016-04-12,0.001847,0.001809,0.002599,0.001656,0.000917,-0.013439,-0.000069,0.000345,0.003361,0.002301,...,0.000124,-0.001770,0.003037,0.002947,-0.002769,-0.001814,-0.002842,-0.001022,0.000283,0.000010
2016-04-13,0.002965,0.004006,0.003140,0.002632,0.001331,-0.014636,0.000650,0.000939,0.004439,0.003287,...,0.001432,0.000255,0.003654,0.003553,-0.001833,-0.001435,-0.000983,0.001159,0.001492,0.000010


In [9]:
agree_on_sign=np.sign(returns.iloc[60:,:-1]) == np.sign(return_estimate.iloc[:,:-1])
print("Return predictions have the right sign %.1f%% of the times"%
      (100*agree_on_sign.sum().sum()/(agree_on_sign.shape[0]*(agree_on_sign.shape[1]-1))))

Return predictions have the right sign 50.1% of the times


In [10]:
volume_estimate=volumes.ewm(alpha=0.1, min_periods=60).mean().dropna()
volume_estimate.to_csv(datadir+'ss_volume_estimate.csv.gz', compression='gzip', float_format='%d')
sigma_estimate = returns.ewm(alpha=0.1, min_periods=60).cov().dropna()
sigma_estimate.to_csv(datadir+'ss_sigma_estimate.csv.gz', compression='gzip', float_format='%.3e')
return_estimate.to_hdf(datadir+'ss_model.h5', 'return_estimate')
volume_estimate.to_hdf(datadir+'ss_model.h5', 'volume_estimate')
sigma_estimate.to_hdf(datadir+'ss_model.h5', 'sigma_estimate')